# Movie Rating Prediction with Stacked Autoencoders in PyTorch

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
ratings = pd.read_csv('ratings_small.csv')#, engine = 'python')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
ratings.drop(["timestamp"], axis=1)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [6]:
movieMax = max(ratings["movieId"])
movieMax

163949

In [7]:
userMax = max(ratings["userId"])
userMax

671

In [8]:
user1 = ratings.loc[ratings["userId"] == 4]
user1

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346
...,...,...,...,...
346,4,3251,5.0,949918970
347,4,3255,4.0,949919738
348,4,3263,3.0,949919845
349,4,3265,5.0,949895732


In [9]:
movieMin = min(ratings["movieId"])
movieMin

1

In [10]:
id_movies = ratings.loc[ratings["userId"] == 2]["movieId"]
id_movies

20     10
21     17
22     39
23     47
24     50
     ... 
91    592
92    593
93    616
94    661
95    720
Name: movieId, Length: 76, dtype: int64

In [11]:
id_ratings = list(ratings.loc[ratings["userId"] == 2]["rating"])
id_ratings

[4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 3.0,
 3.0,
 4.0,
 3.0,
 5.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 3.0,
 5.0,
 1.0,
 3.0,
 3.0,
 3.0,
 4.0,
 4.0,
 5.0,
 5.0,
 3.0,
 4.0,
 3.0,
 4.0,
 3.0,
 4.0,
 2.0,
 1.0,
 3.0,
 4.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 3.0,
 3.0,
 3.0,
 3.0,
 2.0,
 3.0,
 4.0,
 3.0,
 4.0,
 2.0,
 4.0,
 3.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 3.0,
 5.0,
 3.0,
 5.0,
 3.0,
 3.0,
 3.0,
 5.0,
 5.0,
 5.0,
 3.0,
 3.0,
 4.0,
 4.0]

In [12]:
def convert(data):
    new_data = []
    for user in range(1, max(ratings["userId"]+1)):
        id_movies = data.loc[ratings["userId"] == user]["movieId"]
        id_ratings = data.loc[ratings["userId"] == user]["rating"]
        userRatings = np.zeros(movieMax)
        userRatings[id_movies-1] = id_ratings
        new_data.append(userRatings)
    return(new_data)

In [13]:
ratings = convert(ratings)

In [14]:
ratings = np.asarray(ratings)
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [15]:
x_train ,x_test = train_test_split(ratings, test_size=0.2)

In [16]:
len(x_train[1])

163949

In [17]:
len(x_test)

135

In [18]:
x_train = torch.FloatTensor(x_train)
x_test= torch.FloatTensor(x_test)

In [19]:
class SAE(nn.Module):
    def __init__(self, nb_movies):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [20]:
sae = SAE(movieMax)

In [21]:
#sae.to(device)

In [22]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [23]:
len(x_train)

536

In [24]:
#x_train.to(device)
#x_test.to(device)
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(len(x_train)):
        #x_train.to(device)
        input1 = Variable(x_train[id_user]).unsqueeze(0)
        target = input1.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input1)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = movieMax/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt((loss.data)*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(3.7701)
epoch: 2 loss: tensor(3.7362)
epoch: 3 loss: tensor(3.6991)
epoch: 4 loss: tensor(3.6607)
epoch: 5 loss: tensor(3.6206)
epoch: 6 loss: tensor(3.5783)
epoch: 7 loss: tensor(3.5331)
epoch: 8 loss: tensor(3.4845)
epoch: 9 loss: tensor(3.4320)
epoch: 10 loss: tensor(3.3751)
epoch: 11 loss: tensor(3.3134)
epoch: 12 loss: tensor(3.2469)
epoch: 13 loss: tensor(3.1760)
epoch: 14 loss: tensor(3.1031)
epoch: 15 loss: tensor(3.0316)
epoch: 16 loss: tensor(2.9639)
epoch: 17 loss: tensor(2.9011)
epoch: 18 loss: tensor(2.8429)
epoch: 19 loss: tensor(2.7890)
epoch: 20 loss: tensor(2.7388)
epoch: 21 loss: tensor(2.6919)
epoch: 22 loss: tensor(2.6478)
epoch: 23 loss: tensor(2.6064)
epoch: 24 loss: tensor(2.5672)
epoch: 25 loss: tensor(2.5302)
epoch: 26 loss: tensor(2.4950)
epoch: 27 loss: tensor(2.4616)
epoch: 28 loss: tensor(2.4297)
epoch: 29 loss: tensor(2.3994)
epoch: 30 loss: tensor(2.3703)
epoch: 31 loss: tensor(2.3425)
epoch: 32 loss: tensor(2.3159)
epoch: 33 loss: t

In [25]:
#sae.to("cpu")

In [27]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(1, len(x_train)):
    input = Variable(x_train[id_user]).unsqueeze(0)
    target = Variable(x_test[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

IndexError: The shape of the mask [163949] at index 0 does not match the shape of the indexed tensor [1, 163949] at index 0